In [1]:
# --- Jupyter cell: run REINFORCE from notebook (no CLI) ---

from pathlib import Path
from datetime import datetime
import os, sys

# 1) Установим пути: ноутбук в AgentsLab/notebooks/, корень — на уровень выше
NB_DIR = Path.cwd()
LAB_ROOT = NB_DIR.parent
SRC_DIR = LAB_ROOT / "src"
CONFIG_DIR = LAB_ROOT / "configs"

# Если пакет не установлен editable-режимом, добавим src в sys.path
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

# 2) Hydra: программная композиция конфига
from hydra import initialize_config_dir, compose
from omegaconf import OmegaConf


Active config:
 rl:
  env:
    id: CartPole-v1
    max_steps: 500
    render_mode: null
  model:
    obs_dim: null
    action_dim: null
    hidden_dim: 128
    activation: tanh
  optimizer:
    name: adam
    lr: 0.005
    weight_decay: 0.0
  training:
    max_episodes: 500
    gamma: 0.99
    normalize_returns: true
    log_every: 10
    eval_every: 50
    save_every: 100
common:
  use_tensorboard: true
seed: 42
device: cpu

[ep 0010] return=12.00 loss=-1.148
[ep 0020] return=12.00 loss=-1.568
[ep 0030] return=18.00 loss=0.406
[ep 0040] return=32.00 loss=-0.354
[ep 0050] return=89.00 loss=-1.866
[ep 0060] return=217.00 loss=-3.832
[ep 0070] return=315.00 loss=-11.663
[ep 0080] return=33.00 loss=-2.595
[ep 0090] return=221.00 loss=6.275
[ep 0100] return=219.00 loss=-6.959
[ep 0110] return=125.00 loss=-4.363
[ep 0120] return=115.00 loss=-1.974
[ep 0130] return=70.00 loss=-4.799
[ep 0140] return=116.00 loss=0.783
[ep 0150] return=198.00 loss=-5.041
[ep 0160] return=267.00 loss=-2.045
[ep

In [ ]:
with initialize_config_dir(config_dir=str(CONFIG_DIR), version_base=None):
    # Можно указать оверрайды прямо здесь строками Hydra (не через CLI):
    overrides = [
        # Примеры (раскомментируйте и меняйте при необходимости):
        # "rl.env.id=CartPole-v1",
        # "rl.optimizer.lr=0.005",
        # "rl.training.max_episodes=200",
        # "device=cpu",  # или "cuda"
    ]
    cfg = compose(config_name="config", overrides=overrides)

# (опционально) Точечные правки конфига уже как Python-объекта:
# cfg.rl.training.max_episodes = 200
# cfg.seed = 123

print("Active config:\n", OmegaConf.to_yaml(cfg, resolve=True))

# 3) Импорт тренинга и запуск
from agentslab.rl.training.train_loop import train

# Создадим "как у Hydra" run-директорию и на время тренировки перейдём в неё
run_dir = LAB_ROOT / "outputs" / datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
os.makedirs(run_dir, exist_ok=True)

orig_cwd = os.getcwd()
try:
    os.chdir(run_dir)
    train(cfg)  # это вызовет обучение, запишет tb/ и checkpoints/ внутрь run_dir
finally:
    os.chdir(orig_cwd)

print(f"\nRun artifacts saved to: {run_dir}")
print(f"TensorBoard logs:       {run_dir/'tb'}")
print(f"Checkpoints:            {run_dir/'checkpoints'/'rl'}")